In [1]:
from keras import backend as K
    
def Precision(y_true, y_pred):
    """精确率"""
    tp= K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))  # true positives
    pp= K.sum(K.round(K.clip(y_pred, 0, 1))) # predicted positives
    precision = tp/ (pp+ K.epsilon())
    return precision

def Recall(y_true, y_pred):
    """召回率"""
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1))) # true positives
    pp = K.sum(K.round(K.clip(y_true, 0, 1))) # possible positives
    recall = tp / (pp + K.epsilon())
    return recall
 
def F1(y_true, y_pred):
    """F1-score"""
    precision = Precision(y_true, y_pred)
    recall = Recall(y_true, y_pred)
    f1 = 2 * ((precision * recall) / (precision + recall + K.epsilon()))
    return f1    

Using TensorFlow backend.


In [2]:
import kashgari
from kashgari.tasks.labeling import BiLSTM_CRF_Model
from kashgari.embeddings import BERTEmbedding, BareEmbedding
from kashgari.embeddings import NumericFeaturesEmbedding, StackedEmbedding

╭─────────────────────────────────────────────────────────────────────────╮
│ ◎ ○ ○ ░░░░░░░░░░░░░░░░░░░░░  Important Message  ░░░░░░░░░░░░░░░░░░░░░░░░│
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│              We renamed again for consistency and clarity.              │
│                   From now on, it is all `kashgari`.                    │
│  Changelog: https://github.com/BrikerMan/Kashgari/releases/tag/v1.0.0   │
│                                                                         │
│         | Backend          | pypi version   | desc           |          │
│         | ---------------- | -------------- | -------------- |          │
│         | TensorFlow 2.x   | kashgari 2.x.x | coming soon    |          │
│         | TensorFlow 1.14+ | kashgari 1.x.x |                |          │
│         | Keras            | kashgari 0.x.x | legacy version |          │
│           

In [3]:
flb2id = {
    'O': 1,
    'Time': 2,
    'Person': 3,
    'Location': 4,
    'Money': 5,
    'Things': 6,
    'Count': 7
}

In [4]:
trig2id = {
    'O': 1,
    'Steal': 2,
    'Draw': 3,
    'Consume': 4,
    'Sale': 5,
    'Volunteer': 6
}

In [5]:
# 初始化训练数据
train_x = []
# train_islb = []
train_flb = []
# train_istrig = []
# train_trig= []
train_steal = []
train_draw = []
train_consume = []
train_sale = []
train_volun = []
train_steal_dis = []
train_draw_dis = []
train_consume_dis = []
train_sale_dis = []
train_volun_dis = []
train_y = []

with open('train_zi.txt', 'r', encoding='utf-8') as f:
    tmp_x = []
    tmp_flb = []
    # tmp_islb = []
    tmp_trig = []
    # tmp_istrig = []
    tmp_steal = []
    tmp_draw = []
    tmp_consume = []
    tmp_sale = []
    tmp_volun = []
    tmp_steal_dis = []
    tmp_draw_dis = []
    tmp_consume_dis = []
    tmp_sale_dis = []
    tmp_volun_dis = []
    tmp_yy = []
    for i in f.readlines():
        if i == '\n':
            train_x.append(tmp_x)
            train_flb.append(tmp_flb)
            # train_islb.append(tmp_islb)
            # train_trig.append(tmp_trig)
            # train_istrig.append(tmp_istrig)
            train_y.append(tmp_yy)
            # 处理事件标签
            if 2 in tmp_trig:
                idx = tmp_trig.index(2)
                cnt = 0
                for i in tmp_flb:
                    if i != 1:
                        tmp_steal.append(2)
                    else:
                        tmp_steal.append(1)
                    tmp_steal_dis.append(abs(idx-cnt)+1)
                    cnt = cnt + 1
            else:
                for i in tmp_flb:
                    tmp_steal.append(1)
                    tmp_steal_dis.append(256)
            train_steal.append(tmp_steal)
            train_steal_dis.append(tmp_steal_dis)
            if 3 in tmp_trig:
                idx = tmp_trig.index(3)
                cnt = 0
                for i in tmp_flb:
                    if i != 1:
                        tmp_draw.append(2)
                    else:
                        tmp_draw.append(1)
                    tmp_draw_dis.append(abs(idx-cnt)+1)
                    cnt = cnt + 1
            else:
                for i in tmp_flb:
                    tmp_draw.append(1)
                    tmp_draw_dis.append(256)
            train_draw.append(tmp_draw)
            train_draw_dis.append(tmp_draw_dis)
            if 4 in tmp_trig:
                idx = tmp_trig.index(4)
                cnt = 0
                for i in tmp_flb:
                    if i != 1:
                        tmp_consume.append(2)
                    else:
                        tmp_consume.append(1)
                    tmp_consume_dis.append(abs(idx-cnt)+1)
                    cnt = cnt + 1
            else:
                for i in tmp_flb:
                    tmp_consume.append(1)
                    tmp_consume_dis.append(256)
            train_consume.append(tmp_consume)
            train_consume_dis.append(tmp_consume_dis)
            if 5 in tmp_trig:
                idx = tmp_trig.index(5)
                cnt = 0
                for i in tmp_flb:
                    if i != 1:
                        tmp_sale.append(2)
                    else:
                        tmp_sale.append(1)
                    tmp_sale_dis.append(abs(idx-cnt)+1)
                    cnt = cnt + 1
            else:
                for i in tmp_flb:
                    tmp_sale.append(1)
                    tmp_sale_dis.append(256)
            train_sale.append(tmp_sale)
            train_sale_dis.append(tmp_sale_dis)
            if 6 in tmp_trig:
                idx = tmp_trig.index(6)
                cnt = 0
                for i in tmp_flb:
                    if i != 1:
                        tmp_volun.append(2)
                    else:
                        tmp_volun.append(1)
                    tmp_volun_dis.append(abs(idx-cnt)+1)
                    cnt = cnt + 1
            else:
                for i in tmp_flb:
                    tmp_volun.append(1)
                    tmp_volun_dis.append(256)
            train_volun.append(tmp_volun)
            train_volun_dis.append(tmp_volun_dis)
            tmp_x = []
            tmp_flb = []
            # tmp_islb = []
            tmp_trig = []
            # tmp_istrig = []
            tmp_steal = []
            tmp_draw = []
            tmp_consume = []
            tmp_sale = []
            tmp_volun = []
            tmp_steal_dis = []
            tmp_draw_dis = []
            tmp_consume_dis = []
            tmp_sale_dis = []
            tmp_volun_dis = []
            tmp_yy = []
        else:
            x, y = i.split(' ')
            tmp_x.append(x)
            tmp_list = y.split('-')
            if len(tmp_list) == 1:
                tmp_flb.append(1)
                # tmp_islb.append(1)
                tmp_trig.append(1)
                # tmp_istrig.append(1)
                tmp_yy.append(tmp_list[0].replace('\n', ''))
            elif len(tmp_list) == 2:
                # tmp_str = tmp_list[0] + '-' + tmp_list[1].replace('\n', '')
                tmp_flb.append(1)
                # tmp_islb.append(1)
                tmp_trig.append(trig2id[tmp_list[1].replace('\n', '')])
                # tmp_istrig.append(2)
                tmp_yy.append('O')
                # 处理label
            else:
                tmp_str = tmp_list[0] + '-' + tmp_list[1].replace('\n', '')
                tmp_flb.append(flb2id[tmp_list[1].replace('\n', '')])
                # tmp_islb.append(2)
                tmp_trig.append(1)
                # tmp_istrig.append(1)
                if len(tmp_list) == 3:
                    tmp_str = tmp_list[0] + '-' + tmp_list[1] +\
                    tmp_list[2].replace('\n', '')
                    tmp_yy.append(tmp_str)
                elif len(tmp_list) == 4:
                    tmp_str = tmp_list[0] + '-' + tmp_list[3].replace('\n', '') +\
                    tmp_list[2]
                    tmp_yy.append(tmp_str)

In [6]:
print(train_x[1110])
print(train_flb[1110])
print(train_steal[1110])
print(train_draw[1110])
print(train_consume[1110])
print(train_sale[1110])
print(train_volun[1110])
print(train_steal_dis[1110])
print(train_draw_dis[1110])
print(train_consume_dis[1110])
print(train_sale_dis[1110])
print(train_volun_dis[1110])
print(train_y[1110])

['3', '、', '2', '0', '1', '3', '年', '1', '2', '月', '1', '0', '日', '下', '午', '，', '被', '告', '人', '卿', '某', '某', '在', '江', '津', '区', '双', '福', '群', '光', '电', '子', '厂', '宿', '舍', 'X', '—', 'X', 'X', '号', '寝', '室', '内', '，', '盗', '走', '同', '寝', '室', '居', '住', '的', '凌', '某', '的', '联', '想', '牌', '笔', '记', '本', '电', '脑', '一', '台', '。']
[1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 3, 3, 3, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 1, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 1]
[1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [7]:
# 初始化测试数据
test_x = []
# train_islb = []
test_flb = []
# train_istrig = []
# train_trig= []
test_steal = []
test_draw = []
test_consume = []
test_sale = []
test_volun = []
test_steal_dis = []
test_draw_dis = []
test_consume_dis = []
test_sale_dis = []
test_volun_dis = []
test_y = []

with open('test_zi.txt', 'r', encoding='utf-8') as f:
    tmp_x = []
    tmp_flb = []
    # tmp_islb = []
    tmp_trig = []
    # tmp_istrig = []
    tmp_steal = []
    tmp_draw = []
    tmp_consume = []
    tmp_sale = []
    tmp_volun = []
    tmp_steal_dis = []
    tmp_draw_dis = []
    tmp_consume_dis = []
    tmp_sale_dis = []
    tmp_volun_dis = []
    tmp_yy = []
    for i in f.readlines():
        if i == '\n':
            test_x.append(tmp_x)
            test_flb.append(tmp_flb)
            # train_islb.append(tmp_islb)
            # train_trig.append(tmp_trig)
            # train_istrig.append(tmp_istrig)
            test_y.append(tmp_yy)
            # 处理事件标签
            if 2 in tmp_trig:
                idx = tmp_trig.index(2)
                cnt = 0
                for i in tmp_flb:
                    if i != 1:
                        tmp_steal.append(2)
                    else:
                        tmp_steal.append(1)
                    tmp_steal_dis.append(abs(idx-cnt)+1)
                    cnt = cnt + 1
            else:
                for i in tmp_flb:
                    tmp_steal.append(1)
                    tmp_steal_dis.append(256)
            test_steal.append(tmp_steal)
            test_steal_dis.append(tmp_steal_dis)
            if 3 in tmp_trig:
                idx = tmp_trig.index(3)
                cnt = 0
                for i in tmp_flb:
                    if i != 1:
                        tmp_draw.append(2)
                    else:
                        tmp_draw.append(1)
                    tmp_draw_dis.append(abs(idx-cnt)+1)
                    cnt = cnt + 1
            else:
                for i in tmp_flb:
                    tmp_draw.append(1)
                    tmp_draw_dis.append(256)
            test_draw.append(tmp_draw)
            test_draw_dis.append(tmp_draw_dis)
            if 4 in tmp_trig:
                idx = tmp_trig.index(4)
                cnt = 0
                for i in tmp_flb:
                    if i != 1:
                        tmp_consume.append(2)
                    else:
                        tmp_consume.append(1)
                    tmp_consume_dis.append(abs(idx-cnt)+1)
                    cnt = cnt + 1
            else:
                for i in tmp_flb:
                    tmp_consume.append(1)
                    tmp_consume_dis.append(256)
            test_consume.append(tmp_consume)
            test_consume_dis.append(tmp_consume_dis)
            if 5 in tmp_trig:
                idx = tmp_trig.index(5)
                cnt = 0
                for i in tmp_flb:
                    if i != 1:
                        tmp_sale.append(2)
                    else:
                        tmp_sale.append(1)
                    tmp_sale_dis.append(abs(idx-cnt)+1)
                    cnt = cnt + 1
            else:
                for i in tmp_flb:
                    tmp_sale.append(1)
                    tmp_sale_dis.append(256)
            test_sale.append(tmp_sale)
            test_sale_dis.append(tmp_sale_dis)
            if 6 in tmp_trig:
                idx = tmp_trig.index(6)
                cnt = 0
                for i in tmp_flb:
                    if i != 1:
                        tmp_volun.append(2)
                    else:
                        tmp_volun.append(1)
                    tmp_volun_dis.append(abs(idx-cnt)+1)
                    cnt = cnt + 1
            else:
                for i in tmp_flb:
                    tmp_volun.append(1)
                    tmp_volun_dis.append(256)
            test_volun.append(tmp_volun)
            test_volun_dis.append(tmp_volun_dis)
            tmp_x = []
            tmp_flb = []
            # tmp_islb = []
            tmp_trig = []
            # tmp_istrig = []
            tmp_steal = []
            tmp_draw = []
            tmp_consume = []
            tmp_sale = []
            tmp_volun = []
            tmp_steal_dis = []
            tmp_draw_dis = []
            tmp_consume_dis = []
            tmp_sale_dis = []
            tmp_volun_dis = []
            tmp_yy = []
        else:
            x, y = i.split(' ')
            tmp_x.append(x)
            tmp_list = y.split('-')
            if len(tmp_list) == 1:
                tmp_flb.append(1)
                # tmp_islb.append(1)
                tmp_trig.append(1)
                # tmp_istrig.append(1)
                tmp_yy.append(tmp_list[0].replace('\n', ''))
            elif len(tmp_list) == 2:
                # tmp_str = tmp_list[0] + '-' + tmp_list[1].replace('\n', '')
                tmp_flb.append(1)
                # tmp_islb.append(1)
                tmp_trig.append(trig2id[tmp_list[1].replace('\n', '')])
                # tmp_istrig.append(2)
                tmp_yy.append('O')
                # 处理label
            else:
                tmp_str = tmp_list[0] + '-' + tmp_list[1].replace('\n', '')
                tmp_flb.append(flb2id[tmp_list[1].replace('\n', '')])
                # tmp_islb.append(2)
                tmp_trig.append(1)
                # tmp_istrig.append(1)
                if len(tmp_list) == 3:
                    tmp_str = tmp_list[0] + '-' + tmp_list[1] +\
                    tmp_list[2].replace('\n', '')
                    tmp_yy.append(tmp_str)
                elif len(tmp_list) == 4:
                    tmp_str = tmp_list[0] + '-' + tmp_list[3].replace('\n', '') +\
                    tmp_list[2]
                    tmp_yy.append(tmp_str)

In [8]:
# 初始化验证数据
dev_x = []
# train_islb = []
dev_flb = []
# train_istrig = []
# train_trig= []
dev_steal = []
dev_draw = []
dev_consume = []
dev_sale = []
dev_volun = []
dev_steal_dis = []
dev_draw_dis = []
dev_consume_dis = []
dev_sale_dis = []
dev_volun_dis = []
dev_y = []

with open('dev_zi.txt', 'r', encoding='utf-8') as f:
    tmp_x = []
    tmp_flb = []
    # tmp_islb = []
    tmp_trig = []
    # tmp_istrig = []
    tmp_steal = []
    tmp_draw = []
    tmp_consume = []
    tmp_sale = []
    tmp_volun = []
    tmp_steal_dis = []
    tmp_draw_dis = []
    tmp_consume_dis = []
    tmp_sale_dis = []
    tmp_volun_dis = []
    tmp_yy = []
    for i in f.readlines():
        if i == '\n':
            dev_x.append(tmp_x)
            dev_flb.append(tmp_flb)
            # train_islb.append(tmp_islb)
            # train_trig.append(tmp_trig)
            # train_istrig.append(tmp_istrig)
            dev_y.append(tmp_yy)
            # 处理事件标签
            if 2 in tmp_trig:
                idx = tmp_trig.index(2)
                cnt = 0
                for i in tmp_flb:
                    if i != 1:
                        tmp_steal.append(2)
                    else:
                        tmp_steal.append(1)
                    tmp_steal_dis.append(abs(idx-cnt)+1)
                    cnt = cnt + 1
            else:
                for i in tmp_flb:
                    tmp_steal.append(1)
                    tmp_steal_dis.append(256)
            dev_steal.append(tmp_steal)
            dev_steal_dis.append(tmp_steal_dis)
            if 3 in tmp_trig:
                idx = tmp_trig.index(3)
                cnt = 0
                for i in tmp_flb:
                    if i != 1:
                        tmp_draw.append(2)
                    else:
                        tmp_draw.append(1)
                    tmp_draw_dis.append(abs(idx-cnt)+1)
                    cnt = cnt + 1
            else:
                for i in tmp_flb:
                    tmp_draw.append(1)
                    tmp_draw_dis.append(256)
            dev_draw.append(tmp_draw)
            dev_draw_dis.append(tmp_draw_dis)
            if 4 in tmp_trig:
                idx = tmp_trig.index(4)
                cnt = 0
                for i in tmp_flb:
                    if i != 1:
                        tmp_consume.append(2)
                    else:
                        tmp_consume.append(1)
                    tmp_consume_dis.append(abs(idx-cnt)+1)
                    cnt = cnt + 1
            else:
                for i in tmp_flb:
                    tmp_consume.append(1)
                    tmp_consume_dis.append(256)
            dev_consume.append(tmp_consume)
            dev_consume_dis.append(tmp_consume_dis)
            if 5 in tmp_trig:
                idx = tmp_trig.index(5)
                cnt = 0
                for i in tmp_flb:
                    if i != 1:
                        tmp_sale.append(2)
                    else:
                        tmp_sale.append(1)
                    tmp_sale_dis.append(abs(idx-cnt)+1)
                    cnt = cnt + 1
            else:
                for i in tmp_flb:
                    tmp_sale.append(1)
                    tmp_sale_dis.append(256)
            dev_sale.append(tmp_sale)
            dev_sale_dis.append(tmp_sale_dis)
            if 6 in tmp_trig:
                idx = tmp_trig.index(6)
                cnt = 0
                for i in tmp_flb:
                    if i != 1:
                        tmp_volun.append(2)
                    else:
                        tmp_volun.append(1)
                    tmp_volun_dis.append(abs(idx-cnt)+1)
                    cnt = cnt + 1
            else:
                for i in tmp_flb:
                    tmp_volun.append(1)
                    tmp_volun_dis.append(256)
            dev_volun.append(tmp_volun)
            dev_volun_dis.append(tmp_volun_dis)
            tmp_x = []
            tmp_flb = []
            # tmp_islb = []
            tmp_trig = []
            # tmp_istrig = []
            tmp_steal = []
            tmp_draw = []
            tmp_consume = []
            tmp_sale = []
            tmp_volun = []
            tmp_steal_dis = []
            tmp_draw_dis = []
            tmp_consume_dis = []
            tmp_sale_dis = []
            tmp_volun_dis = []
            tmp_yy = []
        else:
            x, y = i.split(' ')
            tmp_x.append(x)
            tmp_list = y.split('-')
            if len(tmp_list) == 1:
                tmp_flb.append(1)
                # tmp_islb.append(1)
                tmp_trig.append(1)
                # tmp_istrig.append(1)
                tmp_yy.append(tmp_list[0].replace('\n', ''))
            elif len(tmp_list) == 2:
                # tmp_str = tmp_list[0] + '-' + tmp_list[1].replace('\n', '')
                tmp_flb.append(1)
                # tmp_islb.append(1)
                tmp_trig.append(trig2id[tmp_list[1].replace('\n', '')])
                # tmp_istrig.append(2)
                tmp_yy.append('O')
                # 处理label
            else:
                tmp_str = tmp_list[0] + '-' + tmp_list[1].replace('\n', '')
                tmp_flb.append(flb2id[tmp_list[1].replace('\n', '')])
                # tmp_islb.append(2)
                tmp_trig.append(1)
                # tmp_istrig.append(1)
                if len(tmp_list) == 3:
                    tmp_str = tmp_list[0] + '-' + tmp_list[1] +\
                    tmp_list[2].replace('\n', '')
                    tmp_yy.append(tmp_str)
                elif len(tmp_list) == 4:
                    tmp_str = tmp_list[0] + '-' + tmp_list[3].replace('\n', '') +\
                    tmp_list[2]
                    tmp_yy.append(tmp_str)

In [9]:
bert_embed = BERTEmbedding('chinese_L-12_H-768_A-12',
                           task=kashgari.LABELING,
                           sequence_length=256)

In [10]:
text_embedding = BareEmbedding(task=kashgari.LABELING,
                               sequence_length=256)
flb_embedding = NumericFeaturesEmbedding(feature_count=7,
                                         feature_name='flb',
                                         sequence_length=256)

islb_embedding = NumericFeaturesEmbedding(feature_count=2,
                                         feature_name='islb',
                                         sequence_length=256)

trig_embedding = NumericFeaturesEmbedding(feature_count=6,
                                         feature_name='trig',
                                         sequence_length=256)

istrig_embedding = NumericFeaturesEmbedding(feature_count=2,
                                         feature_name='istrig',
                                         sequence_length=256)

steal_embedding = NumericFeaturesEmbedding(feature_count=2,
                                         feature_name='steal',
                                         sequence_length=256)

draw_embedding = NumericFeaturesEmbedding(feature_count=2,
                                         feature_name='draw',
                                         sequence_length=256)

consume_embedding = NumericFeaturesEmbedding(feature_count=2,
                                         feature_name='consume',
                                         sequence_length=256)

sale_embedding = NumericFeaturesEmbedding(feature_count=2,
                                         feature_name='sale',
                                         sequence_length=256)

volun_embedding = NumericFeaturesEmbedding(feature_count=2,
                                         feature_name='volun',
                                         sequence_length=256)

steal_dis_embedding = NumericFeaturesEmbedding(feature_count=256,
                                         feature_name='steal_dis',
                                         sequence_length=256)

draw_dis_embedding = NumericFeaturesEmbedding(feature_count=256,
                                         feature_name='draw_dis',
                                         sequence_length=256)

consume_dis_embedding = NumericFeaturesEmbedding(feature_count=256,
                                         feature_name='consume_dis',
                                         sequence_length=256)

sale_dis_embedding = NumericFeaturesEmbedding(feature_count=256,
                                         feature_name='sale_dis',
                                         sequence_length=256)

volun_dis_embedding = NumericFeaturesEmbedding(feature_count=256,
                                         feature_name='volun_dis',
                                         sequence_length=256)

In [11]:
stack_embedding = StackedEmbedding([
    bert_embed,
    flb_embedding,
    steal_embedding,
    draw_embedding,
    consume_embedding,
    sale_embedding,
    volun_embedding,
    steal_dis_embedding,
    draw_dis_embedding,
    consume_dis_embedding,
    sale_dis_embedding,
    volun_dis_embedding
])

In [12]:
train_fx = (train_x, train_flb, train_steal, train_draw, train_consume, train_sale, train_volun,
            train_steal_dis, train_draw_dis, train_consume_dis, train_sale_dis, train_volun_dis)
test_fx = (test_x, test_flb, test_steal, test_draw, test_consume, test_sale, test_volun,
           test_steal_dis, test_draw_dis, test_consume_dis, test_sale_dis, test_volun_dis)
dev_fx = (dev_x, dev_flb, dev_steal, dev_draw, dev_consume, dev_sale, dev_volun,
          dev_steal_dis, dev_draw_dis, train_consume_dis, train_sale_dis, train_volun_dis)

In [13]:
# stack_embedding.analyze_corpus(train_fx, train_y)

In [14]:
ten_fold_x = train_x + dev_x
ten_fold_flb = train_flb + dev_flb
ten_fold_steal = train_steal + dev_steal
ten_fold_draw = train_draw + dev_draw
ten_fold_consume = train_consume + dev_consume
ten_fold_sale = train_sale + dev_sale
ten_fold_volun = train_volun + dev_volun
ten_fold_steal_dis = train_steal_dis + dev_steal_dis
ten_fold_draw_dis = train_draw_dis + dev_draw_dis
ten_fold_consume_dis = train_consume_dis + dev_consume_dis
ten_fold_sale_dis = train_sale_dis + dev_sale_dis
ten_fold_volun_dis = train_volun_dis + dev_volun_dis
ten_fold_y = train_y + dev_y

In [15]:
print(len(ten_fold_x))

5171


In [16]:
model = BiLSTM_CRF_Model(embedding=stack_embedding)

In [17]:
kashgari.config.use_cudnn_cell = True

In [18]:
model.build_model(train_fx,
                  train_y,
                  dev_fx,
                  dev_y)
model.compile_model(metrics=[F1])

Model: "model_19"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 256)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 256)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 256, 768), ( 16226304    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 256, 768)     1536        Input-Segment[0][0]              
___________________________________________________________________________________________

In [ ]:
model.fit(train_fx,
          train_y,
          dev_fx,
          dev_y,
          epochs=1,
          batch_size=64)

 4/74 [>.............................] - ETA: 4:04 - loss: 424.0665 - F1: 0.1567

In [19]:
for j in range(30):
    for i in range(10):
        if i == 0:
            tt_x = ten_fold_x[0:4500]; td_x = ten_fold_x[4500:];
            tt_flb = ten_fold_flb[0:4500]; td_flb = ten_fold_flb[4500:];
            tt_steal = ten_fold_steal[0:4500]; td_steal = ten_fold_steal[4500:];
            tt_draw = ten_fold_draw[0:4500]; td_draw = ten_fold_draw[4500:];
            tt_consume = ten_fold_consume[0:4500]; td_consume = ten_fold_consume[4500:];
            tt_sale = ten_fold_sale[0:4500]; td_sale = ten_fold_sale[4500:];
            tt_volun = ten_fold_volun[0:4500]; td_volun = ten_fold_volun[4500:];
            tt_steal_dis = ten_fold_steal_dis[0:4500]; td_steal_dis = ten_fold_steal_dis[4500:];
            tt_draw_dis = ten_fold_draw_dis[0:4500]; td_draw_dis = ten_fold_draw_dis[4500:];
            tt_consume_dis = ten_fold_consume_dis[0:4500]; td_consume_dis = ten_fold_consume_dis[4500:];
            tt_sale_dis = ten_fold_sale_dis[0:4500]; td_sale_dis = ten_fold_sale_dis[4500:];
            tt_volun_dis = ten_fold_volun_dis[0:4500]; td_volun_dis = ten_fold_volun_dis[4500:];
            tt_y = ten_fold_y[0:4500]; td_y = ten_fold_y[4500:]
            tt_fx = (tt_x, tt_flb, tt_steal, tt_draw, tt_consume, tt_sale, tt_volun,
                     tt_steal_dis, tt_draw_dis, tt_consume_dis, tt_sale_dis, tt_volun_dis)
            td_fx = (td_x, td_flb, td_steal, td_draw, td_consume, td_sale, td_volun,
                     td_steal_dis, td_draw_dis, td_consume_dis, td_sale_dis, td_volun_dis)
            model.fit(tt_fx,
                      tt_y,
                      td_fx,
                      td_y,
                      epochs=1,
                      batch_size=64)
        elif i == 1:
            tt_x = ten_fold_x[0:4000]+ten_fold_x[4500:]; td_x = ten_fold_x[4000:4500];
            tt_flb = ten_fold_flb[0:4000]+ten_fold_flb[4500:]; td_flb = ten_fold_flb[4000:4500];
            tt_steal = ten_fold_steal[0:4000]+ten_fold_steal[4500:]; td_steal = ten_fold_steal[4000:4500];
            tt_draw = ten_fold_draw[0:4000]+ten_fold_draw[4500:]; td_draw = ten_fold_draw[4000:4500];
            tt_consume = ten_fold_consume[0:4000]+ten_fold_consume[4500:]; td_consume = ten_fold_consume[4000:4500];
            tt_sale = ten_fold_sale[0:4000]+ten_fold_sale[4500:]; td_sale = ten_fold_sale[4000:4500];
            tt_volun = ten_fold_volun[0:4000]+ten_fold_volun[4500:]; td_volun = ten_fold_volun[4000:4500];
            tt_steal_dis = ten_fold_steal_dis[0:4000]+ten_fold_steal_dis[4500:]; td_steal_dis = ten_fold_steal_dis[4000:4500];
            tt_draw_dis = ten_fold_draw_dis[0:4000]+ten_fold_draw_dis[4500:]; td_draw_dis = ten_fold_draw_dis[4000:4500];
            tt_consume_dis = ten_fold_consume_dis[0:4000]+ten_fold_consume_dis[4500:]; td_consume_dis = ten_fold_consume_dis[4000:4500];
            tt_sale_dis = ten_fold_sale_dis[0:4000]+ten_fold_sale_dis[4500:]; td_sale_dis = ten_fold_sale_dis[4000:4500];
            tt_volun_dis = ten_fold_volun_dis[0:4000]+ten_fold_volun_dis[4500:]; td_volun_dis = ten_fold_volun_dis[4000:4500];
            tt_y = ten_fold_y[0:4000]+ten_fold_y[4500:]; td_y = ten_fold_y[4000:4500]
            tt_fx = (tt_x, tt_flb, tt_steal, tt_draw, tt_consume, tt_sale, tt_volun,
                     tt_steal_dis, tt_draw_dis, tt_consume_dis, tt_sale_dis, tt_volun_dis)
            td_fx = (td_x, td_flb, td_steal, td_draw, td_consume, td_sale, td_volun,
                     td_steal_dis, td_draw_dis, td_consume_dis, td_sale_dis, td_volun_dis)
            model.fit(tt_fx,
                      tt_y,
                      td_fx,
                      td_y,
                      epochs=1,
                      batch_size=64)
        elif i == 2:
            tt_x = ten_fold_x[0:3500]+ten_fold_x[4000:]; td_x = ten_fold_x[3500:4000];
            tt_flb = ten_fold_flb[0:3500]+ten_fold_flb[4000:]; td_flb = ten_fold_flb[3500:4000];
            tt_steal = ten_fold_steal[0:3500]+ten_fold_steal[4000:]; td_steal = ten_fold_steal[3500:4000];
            tt_draw = ten_fold_draw[0:3500]+ten_fold_draw[4000:]; td_draw = ten_fold_draw[3500:4000];
            tt_consume = ten_fold_consume[0:3500]+ten_fold_consume[4000:]; td_consume = ten_fold_consume[3500:4000];
            tt_sale = ten_fold_sale[0:3500]+ten_fold_sale[4000:]; td_sale = ten_fold_sale[3500:4000];
            tt_volun = ten_fold_volun[0:3500]+ten_fold_volun[4000:]; td_volun = ten_fold_volun[3500:4000];
            tt_steal_dis = ten_fold_steal_dis[0:3500]+ten_fold_steal_dis[4000:]; td_steal_dis = ten_fold_steal_dis[3500:4000];
            tt_draw_dis = ten_fold_draw_dis[0:3500]+ten_fold_draw_dis[4000:]; td_draw_dis = ten_fold_draw_dis[3500:4000];
            tt_consume_dis = ten_fold_consume_dis[0:3500]+ten_fold_consume_dis[4000:]; td_consume_dis = ten_fold_consume_dis[3500:4000];
            tt_sale_dis = ten_fold_sale_dis[0:3500]+ten_fold_sale_dis[4000:]; td_sale_dis = ten_fold_sale_dis[3500:4000];
            tt_volun_dis = ten_fold_volun_dis[0:3500]+ten_fold_volun_dis[4000:]; td_volun_dis = ten_fold_volun_dis[3500:4000];
            tt_y = ten_fold_y[0:3500]+ten_fold_y[4000:]; td_y = ten_fold_y[3500:4000]
            tt_fx = (tt_x, tt_flb, tt_steal, tt_draw, tt_consume, tt_sale, tt_volun,
                     tt_steal_dis, tt_draw_dis, tt_consume_dis, tt_sale_dis, tt_volun_dis)
            td_fx = (td_x, td_flb, td_steal, td_draw, td_consume, td_sale, td_volun,
                     td_steal_dis, td_draw_dis, td_consume_dis, td_sale_dis, td_volun_dis)
            model.fit(tt_fx,
                      tt_y,
                      td_fx,
                      td_y,
                      epochs=1,
                      batch_size=64)
        elif i == 3:
            tt_x = ten_fold_x[0:3000]+ten_fold_x[3500:]; td_x = ten_fold_x[3000:3500];
            tt_flb = ten_fold_flb[0:3000]+ten_fold_flb[3500:]; td_flb = ten_fold_flb[3000:3500];
            tt_steal = ten_fold_steal[0:3000]+ten_fold_steal[3500:]; td_steal = ten_fold_steal[3000:3500];
            tt_draw = ten_fold_draw[0:3000]+ten_fold_draw[3500:]; td_draw = ten_fold_draw[3000:3500];
            tt_consume = ten_fold_consume[0:3000]+ten_fold_consume[3500:]; td_consume = ten_fold_consume[3000:3500];
            tt_sale = ten_fold_sale[0:3000]+ten_fold_sale[3500:]; td_sale = ten_fold_sale[3000:3500];
            tt_volun = ten_fold_volun[0:3000]+ten_fold_volun[3500:]; td_volun = ten_fold_volun[3000:3500];
            tt_steal_dis = ten_fold_steal_dis[0:3000]+ten_fold_steal_dis[3500:]; td_steal_dis = ten_fold_steal_dis[3000:3500];
            tt_draw_dis = ten_fold_draw_dis[0:3000]+ten_fold_draw_dis[3500:]; td_draw_dis = ten_fold_draw_dis[3000:3500];
            tt_consume_dis = ten_fold_consume_dis[0:3000]+ten_fold_consume_dis[3500:]; td_consume_dis = ten_fold_consume_dis[3000:3500];
            tt_sale_dis = ten_fold_sale_dis[0:3000]+ten_fold_sale_dis[3500:]; td_sale_dis = ten_fold_sale_dis[3000:3500];
            tt_volun_dis = ten_fold_volun_dis[0:3000]+ten_fold_volun_dis[3500:]; td_volun_dis = ten_fold_volun_dis[3000:3500];
            tt_y = ten_fold_y[0:3000]+ten_fold_y[3500:]; td_y = ten_fold_y[3000:3500]
            tt_fx = (tt_x, tt_flb, tt_steal, tt_draw, tt_consume, tt_sale, tt_volun,
                     tt_steal_dis, tt_draw_dis, tt_consume_dis, tt_sale_dis, tt_volun_dis)
            td_fx = (td_x, td_flb, td_steal, td_draw, td_consume, td_sale, td_volun,
                     td_steal_dis, td_draw_dis, td_consume_dis, td_sale_dis, td_volun_dis)
            model.fit(tt_fx,
                      tt_y,
                      td_fx,
                      td_y,
                      epochs=1,
                      batch_size=64)
        elif i == 4:
            tt_x = ten_fold_x[0:2500]+ten_fold_x[3000:]; td_x = ten_fold_x[2500:3000];
            tt_flb = ten_fold_flb[0:2500]+ten_fold_flb[3000:]; td_flb = ten_fold_flb[2500:3000];
            tt_steal = ten_fold_steal[0:2500]+ten_fold_steal[3000:]; td_steal = ten_fold_steal[2500:3000];
            tt_draw = ten_fold_draw[0:2500]+ten_fold_draw[3000:]; td_draw = ten_fold_draw[2500:3000];
            tt_consume = ten_fold_consume[0:2500]+ten_fold_consume[3000:]; td_consume = ten_fold_consume[2500:3000];
            tt_sale = ten_fold_sale[0:2500]+ten_fold_sale[3000:]; td_sale = ten_fold_sale[2500:3000];
            tt_volun = ten_fold_volun[0:2500]+ten_fold_volun[3000:]; td_volun = ten_fold_volun[2500:3000];
            tt_steal_dis = ten_fold_steal_dis[0:2500]+ten_fold_steal_dis[3000:]; td_steal_dis = ten_fold_steal_dis[2500:3000];
            tt_draw_dis = ten_fold_draw_dis[0:2500]+ten_fold_draw_dis[3000:]; td_draw_dis = ten_fold_draw_dis[2500:3000];
            tt_consume_dis = ten_fold_consume_dis[0:2500]+ten_fold_consume_dis[3000:]; td_consume_dis = ten_fold_consume_dis[2500:3000];
            tt_sale_dis = ten_fold_sale_dis[0:2500]+ten_fold_sale_dis[3000:]; td_sale_dis = ten_fold_sale_dis[2500:3000];
            tt_volun_dis = ten_fold_volun_dis[0:2500]+ten_fold_volun_dis[3000:]; td_volun_dis = ten_fold_volun_dis[2500:3000];
            tt_y = ten_fold_y[0:2500]+ten_fold_y[3000:]; td_y = ten_fold_y[2500:3000]
            tt_fx = (tt_x, tt_flb, tt_steal, tt_draw, tt_consume, tt_sale, tt_volun,
                     tt_steal_dis, tt_draw_dis, tt_consume_dis, tt_sale_dis, tt_volun_dis)
            td_fx = (td_x, td_flb, td_steal, td_draw, td_consume, td_sale, td_volun,
                     td_steal_dis, td_draw_dis, td_consume_dis, td_sale_dis, td_volun_dis)
            model.fit(tt_fx,
                      tt_y,
                      td_fx,
                      td_y,
                      epochs=1,
                      batch_size=64)
        elif i == 5:
            tt_x = ten_fold_x[0:2000]+ten_fold_x[2500:]; td_x = ten_fold_x[2000:2500];
            tt_flb = ten_fold_flb[0:2000]+ten_fold_flb[2500:]; td_flb = ten_fold_flb[2000:2500];
            tt_steal = ten_fold_steal[0:2000]+ten_fold_steal[2500:]; td_steal = ten_fold_steal[2000:2500];
            tt_draw = ten_fold_draw[0:2000]+ten_fold_draw[2500:]; td_draw = ten_fold_draw[2000:2500];
            tt_consume = ten_fold_consume[0:2000]+ten_fold_consume[2500:]; td_consume = ten_fold_consume[2000:2500];
            tt_sale = ten_fold_sale[0:2000]+ten_fold_sale[2500:]; td_sale = ten_fold_sale[2000:2500];
            tt_volun = ten_fold_volun[0:2000]+ten_fold_volun[2500:]; td_volun = ten_fold_volun[2000:2500];
            tt_steal_dis = ten_fold_steal_dis[0:2000]+ten_fold_steal_dis[2500:]; td_steal_dis = ten_fold_steal_dis[2000:2500];
            tt_draw_dis = ten_fold_draw_dis[0:2000]+ten_fold_draw_dis[2500:]; td_draw_dis = ten_fold_draw_dis[2000:2500];
            tt_consume_dis = ten_fold_consume_dis[0:2000]+ten_fold_consume_dis[2500:]; td_consume_dis = ten_fold_consume_dis[2000:2500];
            tt_sale_dis = ten_fold_sale_dis[0:2000]+ten_fold_sale_dis[2500:]; td_sale_dis = ten_fold_sale_dis[2000:2500];
            tt_volun_dis = ten_fold_volun_dis[0:2000]+ten_fold_volun_dis[2500:]; td_volun_dis = ten_fold_volun_dis[2000:2500];
            tt_y = ten_fold_y[0:2000]+ten_fold_y[2500:]; td_y = ten_fold_y[2000:2500]
            tt_fx = (tt_x, tt_flb, tt_steal, tt_draw, tt_consume, tt_sale, tt_volun,
                     tt_steal_dis, tt_draw_dis, tt_consume_dis, tt_sale_dis, tt_volun_dis)
            td_fx = (td_x, td_flb, td_steal, td_draw, td_consume, td_sale, td_volun,
                     td_steal_dis, td_draw_dis, td_consume_dis, td_sale_dis, td_volun_dis)
            model.fit(tt_fx,
                      tt_y,
                      td_fx,
                      td_y,
                      epochs=1,
                      batch_size=64)
        elif i == 6:
            tt_x = ten_fold_x[0:1500]+ten_fold_x[2000:]; td_x = ten_fold_x[1500:2000];
            tt_flb = ten_fold_flb[0:1500]+ten_fold_flb[2000:]; td_flb = ten_fold_flb[1500:2000];
            tt_steal = ten_fold_steal[0:1500]+ten_fold_steal[2000:]; td_steal = ten_fold_steal[1500:2000];
            tt_draw = ten_fold_draw[0:1500]+ten_fold_draw[2000:]; td_draw = ten_fold_draw[1500:2000];
            tt_consume = ten_fold_consume[0:1500]+ten_fold_consume[2000:]; td_consume = ten_fold_consume[1500:2000];
            tt_sale = ten_fold_sale[0:1500]+ten_fold_sale[2000:]; td_sale = ten_fold_sale[1500:2000];
            tt_volun = ten_fold_volun[0:1500]+ten_fold_volun[2000:]; td_volun = ten_fold_volun[1500:2000];
            tt_steal_dis = ten_fold_steal_dis[0:1500]+ten_fold_steal_dis[2000:]; td_steal_dis = ten_fold_steal_dis[1500:2000];
            tt_draw_dis = ten_fold_draw_dis[0:1500]+ten_fold_draw_dis[2000:]; td_draw_dis = ten_fold_draw_dis[1500:2000];
            tt_consume_dis = ten_fold_consume_dis[0:1500]+ten_fold_consume_dis[2000:]; td_consume_dis = ten_fold_consume_dis[1500:2000];
            tt_sale_dis = ten_fold_sale_dis[0:1500]+ten_fold_sale_dis[2000:]; td_sale_dis = ten_fold_sale_dis[1500:2000];
            tt_volun_dis = ten_fold_volun_dis[0:1500]+ten_fold_volun_dis[2000:]; td_volun_dis = ten_fold_volun_dis[1500:2000];
            tt_y = ten_fold_y[0:1500]+ten_fold_y[2000:]; td_y = ten_fold_y[1500:2000]
            tt_fx = (tt_x, tt_flb, tt_steal, tt_draw, tt_consume, tt_sale, tt_volun,
                     tt_steal_dis, tt_draw_dis, tt_consume_dis, tt_sale_dis, tt_volun_dis)
            td_fx = (td_x, td_flb, td_steal, td_draw, td_consume, td_sale, td_volun,
                     td_steal_dis, td_draw_dis, td_consume_dis, td_sale_dis, td_volun_dis)
            model.fit(tt_fx,
                      tt_y,
                      td_fx,
                      td_y,
                      epochs=1,
                      batch_size=64)
        elif i == 7:
            tt_x = ten_fold_x[0:1000]+ten_fold_x[1500:]; td_x = ten_fold_x[1000:1500];
            tt_flb = ten_fold_flb[0:1000]+ten_fold_flb[1500:]; td_flb = ten_fold_flb[1000:1500];
            tt_steal = ten_fold_steal[0:1000]+ten_fold_steal[1500:]; td_steal = ten_fold_steal[1000:1500];
            tt_draw = ten_fold_draw[0:1000]+ten_fold_draw[1500:]; td_draw = ten_fold_draw[1000:1500];
            tt_consume = ten_fold_consume[0:1000]+ten_fold_consume[1500:]; td_consume = ten_fold_consume[1000:1500];
            tt_sale = ten_fold_sale[0:1000]+ten_fold_sale[1500:]; td_sale = ten_fold_sale[1000:1500];
            tt_volun = ten_fold_volun[0:1000]+ten_fold_volun[1500:]; td_volun = ten_fold_volun[1000:1500];
            tt_steal_dis = ten_fold_steal_dis[0:1000]+ten_fold_steal_dis[1500:]; td_steal_dis = ten_fold_steal_dis[1000:1500];
            tt_draw_dis = ten_fold_draw_dis[0:1000]+ten_fold_draw_dis[1500:]; td_draw_dis = ten_fold_draw_dis[1000:1500];
            tt_consume_dis = ten_fold_consume_dis[0:1000]+ten_fold_consume_dis[1500:]; td_consume_dis = ten_fold_consume_dis[1000:1500];
            tt_sale_dis = ten_fold_sale_dis[0:1000]+ten_fold_sale_dis[1500:]; td_sale_dis = ten_fold_sale_dis[1000:1500];
            tt_volun_dis = ten_fold_volun_dis[0:1000]+ten_fold_volun_dis[1500:]; td_volun_dis = ten_fold_volun_dis[1000:1500];
            tt_y = ten_fold_y[0:1000]+ten_fold_y[1500:]; td_y = ten_fold_y[1000:1500]
            tt_fx = (tt_x, tt_flb, tt_steal, tt_draw, tt_consume, tt_sale, tt_volun,
                     tt_steal_dis, tt_draw_dis, tt_consume_dis, tt_sale_dis, tt_volun_dis)
            td_fx = (td_x, td_flb, td_steal, td_draw, td_consume, td_sale, td_volun,
                     td_steal_dis, td_draw_dis, td_consume_dis, td_sale_dis, td_volun_dis)
            model.fit(tt_fx,
                      tt_y,
                      td_fx,
                      td_y,
                      epochs=1,
                      batch_size=64)
        elif i == 8:
            tt_x = ten_fold_x[0:500]+ten_fold_x[1000:]; td_x = ten_fold_x[500:1000];
            tt_flb = ten_fold_flb[0:500]+ten_fold_flb[1000:]; td_flb = ten_fold_flb[500:1000];
            tt_steal = ten_fold_steal[0:500]+ten_fold_steal[1000:]; td_steal = ten_fold_steal[500:1000];
            tt_draw = ten_fold_draw[0:500]+ten_fold_draw[1000:]; td_draw = ten_fold_draw[500:1000];
            tt_consume = ten_fold_consume[0:500]+ten_fold_consume[1000:]; td_consume = ten_fold_consume[500:1000];
            tt_sale = ten_fold_sale[0:500]+ten_fold_sale[1000:]; td_sale = ten_fold_sale[500:1000];
            tt_volun = ten_fold_volun[0:500]+ten_fold_volun[1000:]; td_volun = ten_fold_volun[500:1000];
            tt_steal_dis = ten_fold_steal_dis[0:500]+ten_fold_steal_dis[1000:]; td_steal_dis = ten_fold_steal_dis[500:1000];
            tt_draw_dis = ten_fold_draw_dis[0:500]+ten_fold_draw_dis[1000:]; td_draw_dis = ten_fold_draw_dis[500:1000];
            tt_consume_dis = ten_fold_consume_dis[0:500]+ten_fold_consume_dis[1000:]; td_consume_dis = ten_fold_consume_dis[500:1000];
            tt_sale_dis = ten_fold_sale_dis[0:500]+ten_fold_sale_dis[1000:]; td_sale_dis = ten_fold_sale_dis[500:1000];
            tt_volun_dis = ten_fold_volun_dis[0:500]+ten_fold_volun_dis[1000:]; td_volun_dis = ten_fold_volun_dis[500:1000];
            tt_y = ten_fold_y[0:500]+ten_fold_y[1000:]; td_y = ten_fold_y[500:1000]
            tt_fx = (tt_x, tt_flb, tt_steal, tt_draw, tt_consume, tt_sale, tt_volun,
                     tt_steal_dis, tt_draw_dis, tt_consume_dis, tt_sale_dis, tt_volun_dis)
            td_fx = (td_x, td_flb, td_steal, td_draw, td_consume, td_sale, td_volun,
                     td_steal_dis, td_draw_dis, td_consume_dis, td_sale_dis, td_volun_dis)
            model.fit(tt_fx,
                      tt_y,
                      td_fx,
                      td_y,
                      epochs=1,
                      batch_size=64)
        elif i == 9:
            tt_x = ten_fold_x[500:]; td_x = ten_fold_x[0:500];
            tt_flb = ten_fold_flb[500:]; td_flb = ten_fold_flb[0:500];
            tt_steal = ten_fold_steal[500:]; td_steal = ten_fold_steal[0:500];
            tt_draw = ten_fold_draw[500:]; td_draw = ten_fold_draw[0:500];
            tt_consume = ten_fold_consume[500:]; td_consume = ten_fold_consume[0:500];
            tt_sale = ten_fold_sale[500:]; td_sale = ten_fold_sale[0:500];
            tt_volun = ten_fold_volun[500:]; td_volun = ten_fold_volun[0:500];
            tt_steal_dis = ten_fold_steal_dis[500:]; td_steal_dis = ten_fold_steal_dis[0:500];
            tt_draw_dis = ten_fold_draw_dis[500:]; td_draw_dis = ten_fold_draw_dis[0:500];
            tt_consume_dis = ten_fold_consume_dis[500:]; td_consume_dis = ten_fold_consume_dis[0:500];
            tt_sale_dis = ten_fold_sale_dis[500:]; td_sale_dis = ten_fold_sale_dis[0:500];
            tt_volun_dis = ten_fold_volun_dis[500:]; td_volun_dis = ten_fold_volun_dis[0:500];
            tt_y = ten_fold_y[500:]; td_y = ten_fold_y[0:500]
            tt_fx = (tt_x, tt_flb, tt_steal, tt_draw, tt_consume, tt_sale, tt_volun,
                     tt_steal_dis, tt_draw_dis, tt_consume_dis, tt_sale_dis, tt_volun_dis)
            td_fx = (td_x, td_flb, td_steal, td_draw, td_consume, td_sale, td_volun,
                     td_steal_dis, td_draw_dis, td_consume_dis, td_sale_dis, td_volun_dis)
            model.fit(tt_fx,
                      tt_y,
                      td_fx,
                      td_y,
                      epochs=1,
                      batch_size=64)

73/73 [==============================] - 45s 621ms/step - loss: 0.0017 - F1: 0.0000e+00 - val_loss: 600.2864 - val_F1: 1.0000


In [20]:
model.evaluate(test_fx, test_y)

                 precision    recall  f1-score   support

PersonVolunteer     0.9091    0.1923    0.3175        52
     ThiefSteal     0.8707    0.7804    0.8231       906
    VictimSteal     0.8585    0.6994    0.7708       928
      MoneyDraw     0.7431    0.7085    0.7254       343
   LocationDraw     0.5161    0.5590    0.5367       229
     CountSteal     0.8868    0.6689    0.7625       761
    ThingsSteal     0.7555    0.5234    0.6184      1175
       TimeSale     0.0000    0.0000    0.0000        15
      TimeSteal     0.5727    0.8131    0.6720       717
     MoneySteal     0.1879    0.0793    0.1115       391
      MoneySale     0.0000    0.0000    0.0000        30
   MoneyConsume     0.0000    0.0000    0.0000        32
  LocationSteal     0.4419    0.5069    0.4722       653
     PersonDraw     0.7941    0.2000    0.3195       135
       TimeDraw     0.3333    0.0794    0.1282       126
     ThingsSale     0.5714    0.0833    0.1455        48
  PersonConsume     0.0000    

'                 precision    recall  f1-score   support\n\nPersonVolunteer     0.9091    0.1923    0.3175        52\n     ThiefSteal     0.8707    0.7804    0.8231       906\n    VictimSteal     0.8585    0.6994    0.7708       928\n      MoneyDraw     0.7431    0.7085    0.7254       343\n   LocationDraw     0.5161    0.5590    0.5367       229\n     CountSteal     0.8868    0.6689    0.7625       761\n    ThingsSteal     0.7555    0.5234    0.6184      1175\n       TimeSale     0.0000    0.0000    0.0000        15\n      TimeSteal     0.5727    0.8131    0.6720       717\n     MoneySteal     0.1879    0.0793    0.1115       391\n      MoneySale     0.0000    0.0000    0.0000        30\n   MoneyConsume     0.0000    0.0000    0.0000        32\n  LocationSteal     0.4419    0.5069    0.4722       653\n     PersonDraw     0.7941    0.2000    0.3195       135\n       TimeDraw     0.3333    0.0794    0.1282       126\n     ThingsSale     0.5714    0.0833    0.1455        48\n  PersonCon

In [80]:
model.save('Saved_Model/p2')

In [ ]:
model.info()